In [11]:
import os
import pandas as pd
import altair as alt
import numpy as np

In [12]:
weather_data = pd.read_csv('clean_weather_data.csv')
covid_data = pd.read_csv('clean_covid_data.csv')

In [13]:
weather_data = weather_data.drop(weather_data.columns[0],axis=1)
covid_data = covid_data.drop(covid_data.columns[0],axis=1)

In [14]:
covid_data

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,32,2020-01-22,Washington,US,1/22/2020 17:00,1.0,0.0,0.0
1,71,2020-01-23,Washington,US,1/23/20 17:00,1.0,0.0,0.0
2,120,2020-01-24,Washington,US,1/24/20 17:00,1.0,0.0,0.0
3,162,2020-01-25,Washington,US,1/25/20 17:00,1.0,0.0,0.0
4,163,2020-01-25,Illinois,US,1/25/20 17:00,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...
14870,192089,2020-12-31,Virginia,US,2021-04-02 15:13:53,349584.0,5032.0,0.0
14871,192098,2020-12-31,Washington,US,2021-04-02 15:13:53,246752.0,3461.0,0.0
14872,192101,2020-12-31,West Virginia,US,2021-04-02 15:13:53,85334.0,1338.0,0.0
14873,192103,2020-12-31,Wisconsin,US,2021-04-02 15:13:53,520438.0,5242.0,0.0


In [15]:
weather_data

,date,Average Temperature (deg F),location,fids
0,2020-03-10,61.769231,Alabama,FIPS:01
1,2020-03-11,63.076923,Alabama,FIPS:01
2,2020-03-12,67.384615,Alabama,FIPS:01
3,2020-03-13,66.538462,Alabama,FIPS:01
4,2020-03-14,66.153846,Alabama,FIPS:01
...,...,...,...,...
13001,2020-12-25,27.888889,Wyoming,FIPS:56
13002,2020-12-26,29.411765,Wyoming,FIPS:56
13003,2020-12-27,21.461538,Wyoming,FIPS:56
13004,2020-12-30,14.727273,Wyoming,FIPS:56


In [16]:
#We can compare average temperature to confirmed cases, deaths, and recoveries
merged_df = covid_data.merge(weather_data,how='inner',left_on=['ObservationDate','Province/State'],right_on=['date','location'])
merged_df['date'] = pd.to_datetime(merged_df['date'])
merged_df

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,date,Average Temperature (deg F),location,fids
0,163,2020-01-25,Illinois,US,1/25/20 17:00,1.0,0.0,0.0,2020-01-25,33.600000,Illinois,FIPS:17
1,208,2020-01-26,Illinois,US,1/26/20 16:00,1.0,0.0,0.0,2020-01-26,33.580000,Illinois,FIPS:17
2,209,2020-01-26,California,US,1/26/20 16:00,2.0,0.0,0.0,2020-01-26,45.983280,California,FIPS:06
3,256,2020-01-27,Illinois,US,1/27/20 23:59,1.0,0.0,0.0,2020-01-27,30.400000,Illinois,FIPS:17
4,257,2020-01-27,California,US,1/27/20 23:59,2.0,0.0,0.0,2020-01-27,45.399615,California,FIPS:06
...,...,...,...,...,...,...,...,...,...,...,...,...
12870,192089,2020-12-31,Virginia,US,2021-04-02 15:13:53,349584.0,5032.0,0.0,2020-12-31,48.800000,Virginia,FIPS:51
12871,192098,2020-12-31,Washington,US,2021-04-02 15:13:53,246752.0,3461.0,0.0,2020-12-31,37.500000,Washington,FIPS:53
12872,192101,2020-12-31,West Virginia,US,2021-04-02 15:13:53,85334.0,1338.0,0.0,2020-12-31,43.600000,West Virginia,FIPS:54
12873,192103,2020-12-31,Wisconsin,US,2021-04-02 15:13:53,520438.0,5242.0,0.0,2020-12-31,19.240000,Wisconsin,FIPS:55


In [17]:
covid_ts = merged_df[['date','location','Confirmed']]
tavg_ts = merged_df[['date','location','Average Temperature (deg F)']]

In [18]:
shift_corrs = {}
shift_corrs_flat = []
shift_vals=[]
#print(covid_ts['location'].unique().shape)
for location in covid_ts['location'].unique():
    loc_fil_covid_ts = covid_ts[covid_ts['location']==location][['date','Confirmed']]
    loc_fil_tavg_ts = tavg_ts[tavg_ts['location']==location][['date','Average Temperature (deg F)']]
    for shift_val in range(1,int((loc_fil_covid_ts['date'].max()-loc_fil_covid_ts['date'].min()).days/2)):
        shifted_covid_cases = loc_fil_covid_ts.shift(periods=shift_val)
        shift_df = pd.concat([shifted_covid_cases,loc_fil_tavg_ts],axis=1)
        if location in shift_corrs:
            shift_corrs[location].append(shift_df.corr().iloc[0,1])
        else:
            shift_corrs[location] = [shift_df.corr().iloc[0,1]]
        shift_corrs_flat.append(shift_df.corr().iloc[0,1])
        shift_vals.append(shift_val)

In [19]:
#print(len(shift_corrs_flat),len(range(1,int((loc_fil_covid_ts['date'].max()-loc_fil_covid_ts['date'].min()).days/2)))*covid_ts['location'].unique().shape[0])
shift_corrs_flat_df = pd.DataFrame({'corr':shift_corrs_flat,'shift_days':shift_vals})

In [20]:
alt.Chart(shift_corrs_flat_df.groupby('shift_days').mean().reset_index()).mark_line().encode(
    x='shift_days',
    y='corr'
)

alt.Chart(...)